In [ ]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.3 MB/s eta 0:00:00


In [ ]:

import hashlib
import json
import base64
import os
import pandas as pd
from time import time
from secrets import token_bytes
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Random import get_random_bytes

# Load dataset from user-uploaded file
def load_dataset(File_path):
    df = pd.read_excel(File_path)  # Assumes dataset is in Excel format
    return df.to_dict(orient='records')  # Convert to list of dictionaries

# Get client name from user
client_name = input("Enter the client's name: ")

data_file = input("Enter the path to the dataset file: ")  # Example: 'Mobile_Forensic_Evidence.xlsx'

evidence_records = load_dataset(data_file)

# Filter evidence for the given client
# Changed 'client_name' to 'Name' - assuming 'Name' is the correct column in the Excel file.
client_evidence = [record for record in evidence_records if record['Name'] == client_name]

if not client_evidence:
    print("No evidence found for this client.")
    exit()

# ... (rest of the code remains the same) ...

# Hashing Function
def sha256_hash(data):
    return hashlib.sha256(json.dumps(data, sort_keys=True).encode()).hexdigest()

# Encryption Functions
def encrypt_xchacha(data, key):
    nonce = os.urandom(12)
    cipher = ChaCha20Poly1305(key)
    ciphertext = cipher.encrypt(nonce, json.dumps(data).encode(), None)
    return base64.b64encode(nonce + ciphertext).decode()

def decrypt_xchacha(enc_data, key):
    raw_data = base64.b64decode(enc_data)
    nonce, ciphertext = raw_data[:12], raw_data[12:]
    cipher = ChaCha20Poly1305(key)
    return json.loads(cipher.decrypt(nonce, ciphertext, None).decode())

# Generate Encryption Key
xchacha_key = token_bytes(32)

evidence_hashes = []
encrypted_evidences = []

for evidence in client_evidence:
    evidence_hash = sha256_hash(evidence)
    encrypted_evidence = encrypt_xchacha(evidence, xchacha_key)
    evidence_hashes.append(evidence_hash)
    encrypted_evidences.append(encrypted_evidence)

# Secure Key Exchange Using RSA
rsa_key = RSA.generate(2048)  # Generate RSA key pair
public_key = rsa_key.publickey()  # Extract public key
private_key = rsa_key.export_key()  # Store private key for later decryption
cipher_rsa = PKCS1_OAEP.new(public_key)
encrypted_xchacha_key = cipher_rsa.encrypt(xchacha_key)  # Encrypt the ChaCha key

# Blockchain Class
class Blockchain:
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        self.new_block(previous_hash="1", proof=100)

    def new_block(self, proof, previous_hash=None):
        block = {
            "index": len(self.chain) + 1,
            "timestamp": time(),
            "transactions": self.current_transactions,
            "proof": proof,
            "previous_hash": previous_hash or self.hash(self.chain[-1]) if self.chain else None,
        }
        self.current_transactions = []
        self.chain.append(block)
        return block

    def new_transaction(self, sender, recipient, evidence_hash, encrypted_data, token):
        self.current_transactions.append({
            "sender": sender,
            "recipient": recipient,
            "evidence_hash": evidence_hash,
            "encrypted_data": encrypted_data,
            "token": token
        })
        return self.last_block["index"] + 1

    def hash(self, block):
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    @property
    def last_block(self):
        return self.chain[-1]

# Initialize Blockchain
blockchain = Blockchain()
security_token = hashlib.sha256("forensic_token".encode()).hexdigest()

# Add Evidence to Blockchain
for i in range(len(client_evidence)):
    blockchain.new_transaction(
        sender="Mobile Device",
        recipient="Forensic Lab",
        evidence_hash=evidence_hashes[i],
        encrypted_data=encrypted_evidences[i],
        token=security_token
    )

blockchain.new_block(proof=12345)

# Display Processed Evidence
print(f"\n📌 Evidence for {client_name}:")
for i, evidence in enumerate(client_evidence):
    print(json.dumps(evidence, indent=4))
    print(f"🔍 Hash: {evidence_hashes[i]}")
    print(f"🔒 Encrypted: {encrypted_evidences[i]}\n")

print("🛡️ Security Token:", security_token)
print("📜 Blockchain Data:")
for block in blockchain.chain:
    print(json.dumps(block, indent=4))

# Display RSA Private Key for Decryption (Keep Secure!)
print("\n🔑 RSA Private Key (Keep Secure!):\n", private_key.decode())

# Display Encrypted ChaCha Key
print("\n🔐 Encrypted XChaCha Key (Using RSA):", base64.b64encode(encrypted_xchacha_key).decode())


Enter the client's name: Harin
Enter the path to the dataset file: /content/Mobile_Forensic_Evidence_Updated.xlsx

📌 Evidence for Harin:
{
    "Name": "Harin",
    "Call Logs": "Call to Bhargav: 11:58 PM",
    "Text Messages": "Kill Uday",
    "Photos": "pic5.bmp, photo2.png",
    "GPS Data": "35.6895\u00b0 N, 139.6917\u00b0 E",
    "Browser History": "www.google.com, www.securityblog.com",
    "Credentials": "admin:admin123, root:toor",
    "Deleted Data": "old_root:toor",
    "Evidence_id": 1,
    "Unnamed: 9": NaN,
    "Case_number": 456803
}
🔍 Hash: eb9d454c22412ed32c8278b38a7cb7fb7d0e9334b1d3baf7ba2f9dde50847ac2
🔒 Encrypted: IZEnW/tpg4nduIJVO/U82zZnJnjA8IDFUMvO3GM20Eeb4qpnQmoXWYlzZlARh/Dws961nsa5e6hvXGR0U+56Dax7m1D+Ijb1ZB7zutE/6QKfI97NiyCfRPIJ4X2mGroY0QjqCXzXFmpe0Nr6WK1n/mRlUATU458JNazVFs79fGJyW6/7zFjhiok4ncK03ydYQkuN1usKUK561RtfTEdwJ2IqrAkuZvjLS8zoSqnp/2/8Eofyo5KPWVWpXTP7+8cmEc/zAakxyLVoxN8XfDobcvqi6T8Zk5Si52o7KBpZsMEEo4wITGIIDwQ7YCHUkW5wxgjC347cVOXJ09sSdexSz++raQIRiXRklzaXm+U1g5